In [25]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

c = 1.0
L = 2.0
J = 100
N = 100
h = L / J
x = np.linspace(0, L, J+1)

def initial_condition_function(x, cond_type):
    if cond_type == "Разрывная функция":
        u0 = np.where((x >= 0.4) & (x <= 0.6), 1.0, 0.0)
    elif cond_type == "Гауссова функция":
        u0 = np.exp(-((x - 0.5) ** 2) / (2 * 0.05**2))
    elif cond_type == "Обрезанный косинус":
        u0 = np.zeros_like(x)
        mask = (x >= 0.3) & (x <= 0.7)
        center = 0.5  
        half_width = 0.2  
        u0[mask] = np.cos((np.pi/2) * (x[mask] - center) / half_width)
        u0 = np.abs(u0)
    else:
        u0 = np.zeros_like(x)
    return u0

def solve_and_plot(sigma, time_index, ic_type):
    tau = sigma * h / c  
    T_effective = N * tau  
    u = np.zeros((N+1, J+1))
    u[0, :] = initial_condition_function(x, ic_type)
    
    for n in range(N):
        for j in range(1, J+1):
            u[n+1, j] = u[n, j] - sigma * (u[n, j] - u[n, j-1])
            
    t = time_index * tau
    x_shifted = (x - c * t) % L
    u_exact = initial_condition_function(x_shifted, ic_type)
    
    plt.figure(figsize=(8, 5))
    plt.plot(x, u[time_index, :], 'b-o', label=f'Численное, t = {t:.3f}')
    plt.plot(x, u_exact, 'r--', label='Теоретическое')
    plt.xlabel('x')
    plt.ylabel('u(x,t)')
    plt.title(f'σ = {sigma:.2f}, T = {T_effective:.3f}')
    plt.legend()
    plt.grid(True)
    plt.ylim(min(u[time_index, :].min(), u_exact.min()) - 0.1, max(u[time_index, :].max(), u_exact.max()) + 0.1)
    plt.show()

interact(solve_and_plot,
         sigma=FloatSlider(min=0.1, max=1.1, step=0.05, value=0.5, description="Число Куранта"),
         time_index=IntSlider(min=0, max=N, step=1, value=0, description="Временной слой"),
         ic_type=Dropdown(options=["Разрывная функция", "Гауссова функция", "Обрезанный косинус"],
                          value="Разрывная функция",
                          description="Начальное условие"));

interactive(children=(FloatSlider(value=0.5, description='Число Куранта', max=1.1, min=0.1, step=0.05), IntSli…